In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
sns.set()
%matplotlib inline

import nengo
import nengo_spa as spa

from nengo_learn_assoc_mem.utils import BasicVecFeed, meg_from_spikes, make_alt_vocab

# Confidence intervals across vocab

# Effect of intercept modification

I expect it to give more distinct trends

# Demonstration of memory operating over time

# Effect of noisier encoders

# Effect of noisier inputs

# Effect of memory as associative

## Can you learn an associative memory off the Ensemble using PES?

## Does this intermediary ensemble change the performance of an engineered associative memory?